In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("TensorFlow:", tf.__version__)

2026-02-09 15:08:41.335195: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770649721.509079      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770649721.568660      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770649722.002521      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770649722.002560      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770649722.002563      55 computation_placer.cc:177] computation placer alr

TensorFlow: 2.19.0


In [2]:
DATA_ROOT = "/kaggle/input/dsgp-dataset-zip/DataSet"
TRAIN_DIR = os.path.join(DATA_ROOT, "/kaggle/input/dsgp-dataset-zip/DataSet/Training")
IMG_SIZE = 224

In [3]:
clf_model = tf.keras.models.load_model("/kaggle/input/tumor-type-classification/tfjs/default/1/brain_tumor_mri_model (1).h5")
clf_model.summary()

I0000 00:00:1770649734.349364      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15511 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 28, 28, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    25,690,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         2,052 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,084,486 (99.50 MB)

 Trainable params: 26,082,500 (99.50 MB)

 Non-trainable params: 1,984 (7.75 KB)

 Optimizer params: 2 (12.00 B)

In [4]:
def load_images(folder):
    images = []
    filenames = []

    for file in os.listdir(folder):
        file_path = os.path.join(folder, file)

        # Skip directories
        if not os.path.isfile(file_path):
            continue

        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)

        # Skip unreadable / non-image files
        if img is None:
            print("Skipping file:", file)
            continue

        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img / 255.0

        images.append(img)
        filenames.append(file)

    return np.array(images), filenames

In [5]:
images, filenames = load_images(TRAIN_DIR)
images = images[..., np.newaxis]

print("Loaded images:", images.shape)

Loaded images: (0, 1)


In [6]:
def grad_cam(model, img_array, class_index, layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, class_index]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]

    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.reduce_max(heatmap)

    return heatmap.numpy()

In [7]:
def grad_cam(model, img_array, class_index, layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, class_index]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]

    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.reduce_max(heatmap)

    return heatmap.numpy()

In [8]:
MENINGIOMA_ID = 1
print("Meningioma class ID:", MENINGIOMA_ID)

Meningioma class ID: 1


In [9]:
print(clf_model.output_shape)

(None, 4)


In [10]:
for layer in clf_model.layers:
    if "conv" in layer.name:
        print(layer.name)

conv2d
conv2d_1
conv2d_2
conv2d_3


In [11]:
LAYER_NAME = "conv2d_3"   # or whatever the last one is

In [12]:
masks = []

for img in images:
    input_img = np.expand_dims(img, axis=0).astype(np.float32)

    heatmap = grad_cam(
        model=clf_model,
        img_array=input_img,
        class_index=MENINGIOMA_ID,
        layer_name=LAYER_NAME
    )

    heatmap = cv2.resize(heatmap, (IMG_SIZE, IMG_SIZE))
    mask = (heatmap > 0.4).astype(np.uint8)  # binary mask
    masks.append(mask)

masks = np.array(masks)[..., np.newaxis]
print("Masks shape:", masks.shape)

Masks shape: (0, 1)


In [13]:
print("Images shape:", images.shape)

Images shape: (0, 1)


In [14]:
print("TRAIN_DIR =", TRAIN_DIR)
print("Contents:", os.listdir(TRAIN_DIR))

TRAIN_DIR = /kaggle/input/dsgp-dataset-zip/DataSet/Training
Contents: ['pituitary', 'notumor', 'meningioma', 'glioma']


In [15]:
MENINGIOMA_DIR = os.path.join(TRAIN_DIR, "meningioma")

print("MENINGIOMA_DIR exists:", os.path.isdir(MENINGIOMA_DIR))
print("Number of files:", len(os.listdir(MENINGIOMA_DIR)))

MENINGIOMA_DIR exists: True
Number of files: 1339


In [16]:
images, filenames = load_images(MENINGIOMA_DIR)
images = images[..., np.newaxis]

print("Loaded images shape:", images.shape)

Loaded images shape: (1339, 224, 224, 1)


In [17]:
IMG_SIZE = 224

def load_images(folder):
    images = []
    filenames = []

    for file in os.listdir(folder):
        if not file.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        path = os.path.join(folder, file)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

        if img is None:
            print("⚠️ Failed to load:", file)
            continue

        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img.astype(np.float32) / 255.0

        images.append(img)
        filenames.append(file)

    return np.array(images), filenames

In [18]:
# Force model to build graph
_ = clf_model.predict(images[:1])

I0000 00:00:1770649749.913676     122 service.cc:152] XLA service 0x78740800d360 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1770649749.913707     122 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1770649750.068154     122 cuda_dnn.cc:529] Loaded cuDNN version 91002


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


I0000 00:00:1770649751.376415     122 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [19]:
import tensorflow as tf
import numpy as np

def grad_cam(model, img_array, class_index, layer_name):
    conv_layer = model.get_layer(layer_name)

    grad_model = tf.keras.models.Model(
        inputs=model.inputs,
        outputs=[conv_layer.output, model.output]  # model.output (NOT model.outputs)
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, class_index]  # now predictions is a tensor

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(conv_outputs * pooled_grads, axis=-1)

    heatmap = tf.maximum(heatmap, 0)
    heatmap /= tf.reduce_max(heatmap) + 1e-8

    return heatmap.numpy()

In [20]:
print(images.shape)

(1339, 224, 224, 1)


In [21]:
import os, cv2
import numpy as np

IMG_SIZE = 224
MENINGIOMA_DIR = os.path.join(TRAIN_DIR, "meningioma")

def load_images(folder):
    images = []
    filenames = []

    for file in os.listdir(folder):
        path = os.path.join(folder, file)

        if not file.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue

        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img / 255.0

        images.append(img)
        filenames.append(file)

    return np.array(images), filenames

In [22]:
images, filenames = load_images(MENINGIOMA_DIR)
images = images[..., np.newaxis]

print("Images shape:", images.shape)

Images shape: (1339, 224, 224, 1)


In [23]:
# Force model graph to build
_ = clf_model.predict(images[:1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


In [24]:
import tensorflow as tf

def grad_cam(model, img_array, class_index, layer_name):
    conv_layer = model.get_layer(layer_name)

    grad_model = tf.keras.models.Model(
        inputs=model.inputs,
        outputs=[conv_layer.output, model.outputs]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, class_index]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(conv_outputs * pooled_grads, axis=-1)

    heatmap = tf.maximum(heatmap, 0)
    heatmap /= tf.reduce_max(heatmap) + 1e-8

    return heatmap.numpy()

In [25]:
MENINGIOMA_ID = clf_model.predict(images[:1]).argmax()
print("Using predicted class ID:", MENINGIOMA_ID)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Using predicted class ID: 1


In [26]:
print("Images shape:", images.shape)
print("Number of images:", len(images))

Images shape: (1339, 224, 224, 1)
Number of images: 1339


In [27]:
def grad_cam(model, img_array, class_index, layer_name):
    conv_layer = model.get_layer(layer_name)

    # Use model.output (singular) to get the tensor, not the list
    grad_model = tf.keras.models.Model(
        inputs=model.inputs,
        outputs=[conv_layer.output, model.output] 
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        # Now 'predictions' is a Tensor, so this slicing works:
        loss = predictions[:, class_index]

    # Calculate gradients of the loss w.r.t. the conv layer output
    grads = tape.gradient(loss, conv_outputs)
    
    # Mean of gradients across height/width (Global Average Pooling)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Weight the channels of the conv output by the pooled gradients
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(conv_outputs * pooled_grads, axis=-1)

    # ReLU and Normalize
    heatmap = tf.maximum(heatmap, 0)
    heatmap /= (tf.reduce_max(heatmap) + 1e-8)

    return heatmap.numpy()

In [29]:
# These should already be set earlier in your notebook
IMG_SIZE = 224
LAYER_NAME = "conv2d_3"          # ← last/biggest conv layer — check your model.summary()
MENINGIOMA_ID = 1                # or whatever index meningioma has in your 4 classes

# Assuming you already have:
# test_images     → shape (N, 224, 224, 1) or (N, 224, 224, 3)
# test_labels     → shape (N,)     integers 0–3
# CLASSES = ['glioma', 'meningioma', 'notumor', 'pituitary']

In [31]:
# ────────────────────────────────────────────────
# 1. Define paths & classes (should already exist, but make sure)
# ────────────────────────────────────────────────

import os
import numpy as np
import cv2

DATA_ROOT = "/kaggle/input/dsgp-dataset-zip/DataSet"
TEST_DIR = os.path.join(DATA_ROOT, "Testing")

CLASSES = ['glioma', 'meningioma', 'notumor', 'pituitary']
CLASS_TO_IDX = {name: idx for idx, name in enumerate(CLASSES)}

IMG_SIZE = 224

# ────────────────────────────────────────────────
# 2. Reuse or slightly improve your load_images function
# ────────────────────────────────────────────────

def load_images_from_folder(folder):
    images = []
    filenames = []
    
    for file in os.listdir(folder):
        if not file.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue
            
        path = os.path.join(folder, file)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        
        if img is None:
            continue
            
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img.astype(np.float32) / 255.0
        
        images.append(img)
        filenames.append(file)
    
    if not images:
        print(f"Warning: No images loaded from {folder}")
        return np.array([]), []
        
    return np.array(images), filenames


# ────────────────────────────────────────────────
# 3. Load ALL test images + labels
# ────────────────────────────────────────────────

test_images_list = []
test_labels_list = []
test_filenames = []

for class_name in CLASSES:
    class_folder = os.path.join(TEST_DIR, class_name)
    if not os.path.isdir(class_folder):
        print(f"Folder not found: {class_folder}")
        continue
        
    imgs, fnames = load_images_from_folder(class_folder)
    if len(imgs) == 0:
        continue
        
    test_images_list.append(imgs)
    test_labels_list.append(np.full(len(imgs), CLASS_TO_IDX[class_name]))
    test_filenames.extend(fnames)

# Combine
if test_images_list:
    test_images = np.concatenate(test_images_list, axis=0)[..., np.newaxis]   # (N, 224, 224, 1)
    test_labels = np.concatenate(test_labels_list)
    print(f"Test set loaded → {len(test_images)} images, shape: {test_images.shape}")
else:
    print("ERROR: No test images were loaded at all!")

Test set loaded → 1826 images, shape: (1826, 224, 224, 1)


In [32]:
# ────────────────────────────────────────────────
# 4. Now it's safe to do dummy prediction
# ────────────────────────────────────────────────

_ = clf_model.predict(test_images[:1], verbose=0)
print("Model graph built successfully (no more 'never called' errors)")

Model graph built successfully (no more 'never called' errors)


In [33]:
print("Does TEST_DIR exist?", os.path.isdir(TEST_DIR))
print("Subfolders:", os.listdir(TEST_DIR) if os.path.isdir(TEST_DIR) else "— folder missing —")

Does TEST_DIR exist? True
Subfolders: ['pituitary', 'notumor', 'meningioma', 'glioma']


In [34]:
print("Variable 'test_images' exists?", 'test_images' in globals())

Variable 'test_images' exists? True
